In [ ]:
from scipy import ndimage




grads = []
for pic in X:
    grads.append(list(np.gradient(pic, 1)) + [ndimage.fourier.fourier_ellipsoid(pic,3).sum(),ndimage.center_of_mass(pic)[0], ndimage.extrema(pic)[3][0], pic.reshape((30,20)).diagonal().sum(), pic.reshape((30,20)).diagonal(offset=1).sum() ] + list(ndimage.gaussian_laplace(pic,2.7))   )#+ list(desc.describe(pic.reshape((30,20)))))
    
grads_test = []
for pic in X_tst:
    grads_test.append(list(np.gradient(pic, 1))+ [ndimage.fourier.fourier_ellipsoid(pic,3).sum(), ndimage.center_of_mass(pic)[0], ndimage.extrema(pic)[2][0], pic.reshape((30,20)).diagonal().sum(), pic.reshape((30,20)).diagonal(offset=1).sum()] + list(ndimage.gaussian_laplace(pic,2.7)) ) # + list(desc.describe(pic.reshape((30,20)))) )
    

    
    
    
    
    
grads = np.array(grads)#.reshape(536,600+1)
grads_test = np.array(grads_test)#.reshape(np.array(grads_test).shape[0],1+np.array(grads_test).shape[2])






from sklearn.model_selection import train_test_split
boosters = []
n_boosters = 1

for i in range(0, n_boosters):
    #Xtr, Xval, Ytr, Yval = train_test_split(grads, np.array(y)-1, test_size=0.2)#,random_state=4455)
    dmat_train = xgb.DMatrix(grads, np.array(y)-1)
    dmat_test = xgb.DMatrix((grads_test), np.array(y_tst)-1)
    #dmat_train = xgb.DMatrix(Xtr,Ytr)
    #dmat_test = xgb.DMatrix(Xval,Yval)
        
    params = {"objective": "binary:logistic",
                
          "eta": 0.001,
          "lambda": 5,
          "alpha": 5,
          "max_depth": 10,
          "eval_metric": "auc", }
    bst = xgb.train(dtrain=dmat_train, params=params, num_boost_round=100000, early_stopping_rounds=50, verbose_eval=100, evals=[(dmat_test, "val_0")])
    
    boosters.append(bst)
    
    
    
    
    
preds = np.array([bst.predict(xgb.DMatrix(grads_test)) for bst in boosters])
preds = np.mean(preds, axis=0)


#p=[]
#threshme(p,bst.predict(xgb.DMatrix(grads_test)),0.5)
#threshme(p,preds,0.5)

#wrong=[]
#eval(np.array(p)-1,(np.array(y_tst)-2)*-1,1,wrong)
